# Direct beam iterations for LoKI

## Introduction

This notebook is used to compute the direct beam function for the LoKI detectors.
It uses data recorded during the detector test at the Larmor instrument.

In [ ]:
import scipp as sc
import sciline
import scippneutron as scn
import plopp as pp
import esssans as sans
from esssans.types import *
from esssans.larmor import DataAsStraws

In [ ]:
%matplotlib widget

## Define reduction parameters

We define a dictionary containing the reduction parameters, with keys and types given by aliases or types defined in `esssans.types`:

In [ ]:
params = {}

params[Filename[SampleRun]] = '60339-2022-02-28_2215.nxs'
params[Filename[SampleTransmissionRun]] = '60394-2022-02-28_2215.nxs'
params[Filename[EmptyBeamRun]] = '60392-2022-02-28_2215.nxs'

params[NeXusMonitorName[Incident]] = 'monitor_1'
params[NeXusMonitorName[Transmission]] = 'monitor_2'

wavelength_min = sc.scalar(1.0, unit='angstrom')
wavelength_max = sc.scalar(13.0, unit='angstrom')
n_wavelength_bins = 300

n_wavelength_bands = 100
sampling_width = sc.scalar(0.2, unit='angstrom')

# Derived params
sampling_half_width = sampling_width * 0.5

wavelength_sampling_points = sc.linspace(
    dim='wavelength',
    start=wavelength_min + sampling_half_width,
    stop=wavelength_max - sampling_half_width,
    num=n_wavelength_bands
)

band_start = wavelength_sampling_points - sampling_half_width
band_end = wavelength_sampling_points + sampling_half_width
wavelength_bins = sc.linspace('wavelength', wavelength_min, wavelength_max, n_wavelength_bins + 1)
wavelength_bands = sc.concat([
    sc.concat([start, end], dim='wavelength')
    for start, end in zip(band_start, band_end)], dim='band')

params[WavelengthBins] = wavelength_bins
params[WavelengthBands] = wavelength_bands

params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.3, num=101, unit='1/angstrom')

# Make a flat direct beam to start with
direct_beam = sc.DataArray(
    data=sc.ones(sizes={'wavelength': n_wavelength_bands}),
    coords={'wavelength': wavelength_sampling_points})

params[DirectBeam] = direct_beam

In [ ]:
params_full = params.copy()
params_full[WavelengthBands] = sc.concat([wavelength_min, wavelength_max], dim='wavelength')

## Create pipeline using Sciline

We use all providers available in `esssans` as well as the `sans2d`-specific providers, which include I/O and mask setup specific to the [Sans2d](https://www.isis.stfc.ac.uk/Pages/sans2d.aspx) instrument:

In [ ]:
providers = sans.providers + sans.larmor.providers

# Pipeline with wavelength bands
pipeline = sciline.Pipeline(providers, params=params)

# Pipeline for the full wavelength range
pipeline_full = sciline.Pipeline(providers, params=params_full)

In [ ]:
pipeline.visualize(IofQ[SampleRun], graph_attr={'rankdir': 'LR'})

In [ ]:
%%time

import numpy as np

niter = 4

for it in range(niter):

    print(f"Iteration {it}")

    iofq_full = pipeline_full.compute(IofQ[SampleRun])
    iofq_slices = pipeline.compute(IofQ[SampleRun])

    eff = []
    for sl in sc.collapse(iofq_slices, keep='Q').values():
        # Try to remove noisy parts as well, maybe criterion based on the size of the error bars?
        sel = (sl.data > 0.) & (sl.data != sc.scalar(np.inf)) & ((sc.variances(sl) / sc.values(sl)).data < 1.0)
        eff.append((sc.values(sl[sel].data).nansum() / sc.values(iofq_full[sel].data).nansum()))

    direct_beam *= sc.concat(eff, dim='wavelength')


In [ ]:
pp.plot({**sc.collapse(iofq_slices, keep='Q'),
         **{'full': iofq_full}}, norm='log', color={'full': 'k'},
        legend=False)

In [ ]:
direct_beam.plot()

In [ ]:
direct_beam